In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
tqdm.pandas()

### Exploratory data analysis of the Lending Club peer-to-peer loan portfolio

Lending Club, founded in 2006, was a pioneering peer-to-peer (P2P) lending platform in the US. It provided a marketplace where individual investors could fund loans directly for borrowers seeking personal loans, debt consolidation, or other financial needs. A snapshot of their lending data taken in April 2019 (featuring originations to the end of 2018) can be obtaing from kaggle:
```
#!/bin/bash
curl -L -o ./lending-club.zip https://www.kaggle.com/api/v1/datasets/download/wordsforthewise/lending-club
unzip lending-club.zip
```

First, let's load the data into a pandas dataframe so we can do some pre-processing so we can use it to train our model.

In [2]:
df = pd.read_csv('../lendingclub/accepted_2007_to_2018Q4.csv.gz', low_memory=False)

Drop loans with missing issue date.

In [3]:
df.issue_d.isna().sum()

33

In [4]:
df = df[df.issue_d.notna()].reset_index(drop=True)

Convert timestamps to appropriate data types.

In [5]:
df.issue_d = pd.to_datetime(df.issue_d, format='%b-%Y')
df.last_pymnt_d = pd.to_datetime(df.last_pymnt_d, format='%b-%Y')

If no payments have been made at all, ensure `last_pymnt_d` is populated even if that's with the date of issue. Bump all payments in the first month of a loan to the following month.

In [6]:
df.last_pymnt_d = df.last_pymnt_d.combine_first(df.issue_d)
df.loc[df.issue_d==df.last_pymnt_d, 'last_pymnt_d'] = df.last_pymnt_d + pd.tseries.offsets.DateOffset(months=1)

Determine date on which snapshot was created.

In [7]:
df['snapshot_d'] = df.last_pymnt_d.max()

### Limitations of a single monthly snapshot

The dataset’s single monthly snapshot limits our ability to analyze loans consistently across their lifecycle—comparing new loans (e.g., 3 months old) with closed ones (e.g., 10+ years old) is like judging a film by one frame. For closed loans, did they repay early, late due to deferments, or because of arrears? For "current" loans, which faced past struggles or exceed their 60-month term? A single snapshot obscures these dynamics. Credit risk modeling demands time series data to fairly compare loans (e.g., 2007 vs. 2017 vintages) at equivalent lifecycle stages and to reconstruct behaviors like: payment momentum, deferment impacts, or hidden delinquency patterns, that define true risk.

To bridge gaps, we’ll simulate a monthly time series spanning the full term of the loan, or to April 2019, whichever comes sooner.

In [8]:
df['term_numeric'] = pd.to_numeric(df.term.str.replace('months', ''), errors='coerce')
df['n_report_d'] = df['snapshot_d'].dt.to_period('M').astype('int64') - df['issue_d'].dt.to_period('M').astype('int64')
df.n_report_d = df[['term_numeric', 'n_report_d']].min(axis=1)

We need to sample `n_report_d` monthly timesteps, starting from the date of issue. Now we determine the total amount paid prior to charge-off.

In [9]:
df['total_pre_chargeoff'] = df.total_pymnt - df.recoveries

### Inflate the dataset to represent the full monthly timeseries from the point of origination up to the true report date

By enumerating a time series of expected installments from each loan’s issue date to the April 2019 snapshot, we create a dynamic timeline of inferred payments.

This basic attribution model serves as a starting point. Using only the issue date, total amount received, and the date and value of the last payment, we create a simple interpolation of payments that respects these 4 facts. 

In [10]:
def basic_pymnt_attr(
    issue_d, 
    n_report_d, 
    last_pymnt_d,
    total_pre_chargeoff,
    last_pymnt_amnt,
    **kwargs):

    # numerate a sequence of report dates from issue to snapshot date
    report_d = pd.date_range(start=issue_d, periods=n_report_d+1, freq='MS', inclusive='right')
    n_pymnt = (report_d < last_pymnt_d).sum()

    # spread the total pymnt made over the vector of months paid
    with np.errstate(divide='ignore', invalid='ignore'):
        pymnt = np.float32(total_pre_chargeoff - last_pymnt_amnt) / n_pymnt
    pymnt = np.full(n_report_d, pymnt)
    pymnt[report_d >= last_pymnt_d] = 0

    # backload the surplus onto the last payment (will be 0 for charged off) and return
    pymnt[report_d == last_pymnt_d] = last_pymnt_amnt
    return pd.Series({
        'report_d': report_d,
        'pymnt': pymnt
    })

In [11]:
df_ts = df.progress_apply(lambda x: basic_pymnt_attr(**x.squeeze()), axis=1).join(df.id)

  0%|          | 0/2260668 [00:00<?, ?it/s]

In [12]:
df_ts.to_json('accepted_2007_to_2018Q4.jsonl.gz', orient='records', lines=True, date_format='iso')

### Select a dataset at random for training the model

This subset of 100,000 records is intended to reduce training time for the model. We can scale this up to the full 2 million records once we're happy with the results.

In [13]:
df_train = df_ts.sample(100_000, random_state=42).reset_index(drop=True)

In [14]:
df_train = df_train.merge(df[['id', 'loan_amnt', 'int_rate', 'installment', 'n_report_d', 'total_pre_chargeoff']])

In [15]:
df_train.to_json('training.jsonl.gz', orient='records', lines=True, date_format='iso')

In [16]:
df_train

,report_d,pymnt,id,loan_amnt,int_rate,installment,n_report_d,total_pre_chargeoff
0,"DatetimeIndex(['2014-01-01', '2014-02-01', '20...","[859.7100446428572, 859.7100446428572, 859.710...",9785323,28000.0,6.62,859.71,36,30949.252158
1,"DatetimeIndex(['2016-06-01', '2016-07-01', '20...","[396.4228210449219, 396.4228210449219, 396.422...",78558461,10650.0,25.11,424.07,34,13702.025250
2,"DatetimeIndex(['2017-07-01', '2017-08-01', '20...","[588.514501953125, 588.514501953125, 588.51450...",110757806,18000.0,10.91,588.54,21,12358.830000
3,"DatetimeIndex(['2017-02-01', '2017-03-01', '20...","[142.36812337239584, 142.36812337239584, 142.3...",96668860,3975.0,17.99,143.69,26,4245.304383
4,"DatetimeIndex(['2017-10-01', '2017-11-01', '20...","[168.30764590992646, 168.30764590992646, 168.3...",118207599,4800.0,16.02,168.81,18,3030.040000
...,...,...,...,...,...,...,...,...
99995,"DatetimeIndex(['2017-09-01', '2017-10-01', '20...","[502.6505533854167, 502.6505533854167, 502.650...",115090136,24000.0,9.44,503.35,19,9551.060000
99996,"DatetimeIndex(['2013-03-01', '2013-04-01', '20...","[438.72006138392857, 438.72006138392857, 438.7...",3365462,13000.0,13.11,438.72,36,15793.502439
99997,"DatetimeIndex(['2015-06-01', '2015-07-01', '20...","[91.3834228515625, 91.3834228515625, 91.383422...",48605201,2500.0,15.61,87.42,36,3116.059464
99998,"DatetimeIndex(['2015-08-01', '2015-09-01', '20...","[223.24749755859375, 223.24749755859375, 223.2...",56117368,14400.0,9.17,300.11,44,1193.100000
